# Smart Beta Portfolio and Portfolio Optimization

In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

^C


### Load Packages

In [1]:
import pandas as pd
import numpy as np
import helper
import project_helper
import project_tests

## Market Data
### Load Data
For this universe of stocks, we'll be selecting large dollar volume stocks. We're using this universe, since it is highly liquid.

In [2]:
df = pd.read_csv('./data/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

### View Data
To see what one of these 2-d matrices looks like, let's take a look at the closing prices matrix.

In [3]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio
build a portfolio using dividend yield to choose the portfolio weights. A portfolio such as this could be incorporated into a smart beta ETF. This portfolio is then compared to a market cap weighted index to see how well it performs. 

In practice, one probably gets the index weights from a data vendor (such as companies that create indices, like MSCI, FTSE, Standard and Poor's). For the sake of practice, a market cap weighted index is simulated.

## Index Weights
The index we'll be using is based on large dollar volume stocks. `generate_dollar_volume_weights` generates the weights for this index. For each date, the weights based on dollar volume traded for that date are generated. For example, assume the following is close prices and volume data:
```
                 Prices
               A         B         ...
2013-07-08     2         2         ...
2013-07-09     5         6         ...
2013-07-10     1         2         ...
2013-07-11     6         5         ...
...            ...       ...       ...

                 Volume
               A         B         ...
2013-07-08     100       340       ...
2013-07-09     240       220       ...
2013-07-10     120       500       ...
2013-07-11     10        100       ...
...            ...       ...       ...
```
The weights created from the function `generate_dollar_volume_weights` should be the following:
```
               A         B         ...
2013-07-08     0.126..   0.194..   ...
2013-07-09     0.759..   0.377..   ...
2013-07-10     0.075..   0.285..   ...
2013-07-11     0.037..   0.142..   ...
...            ...       ...       ...
```

In [4]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    price_volume = close * volume
    dollar_volume_weights = price_volume.div(price_volume.sum(axis=1), axis=0)
    
    return dollar_volume_weights

project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:154: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only

C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:155: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only



### View Data
Let's generate the index weights using `generate_dollar_volume_weights` and view them using a heatmap.

In [5]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

## Portfolio Weights
Having the index weights, next is choosing the portfolio weights based on dividend -normally calculated based on trailing dividend yield. For simplicity the index weights are rather estimated by calculating the total dividend yield over time.

`calculate_dividend_weights` returns the weights for each stock based on its total dividend yield over time. This is similar to generating the weight for the index, but it's using dividend data instead.

For example, assume the following is `dividends` data:
```
                 Prices
               A         B
2013-07-08     0         0
2013-07-09     0         1
2013-07-10     0.5       0
2013-07-11     0         0
2013-07-12     2         0
...            ...       ...
```
The weights created from the function `calculate_dividend_weights` should be the following:
```
               A         B
2013-07-08     NaN       NaN
2013-07-09     0         1
2013-07-10     0.333..   0.666..
2013-07-11     0.333..   0.666..
2013-07-12     0.714..   0.285..
...            ...       ...
```

In [6]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    
    dividend_growth = dividends.cumsum()
    dividend_weights = dividend_growth.div(dividend_growth.sum(axis=1),0)

    return dividend_weights

project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

Tests Passed


C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:154: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only

C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:155: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only



### View Data
Just like the index weights, let's generate the ETF weights and view them using a heatmap.

In [7]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

## Returns
`generate_returns` generates returns data for all the stocks and dates from price data. Since we're not dealing with volatility, we don't have to use ***log returns***.

In [8]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """

    shifted_price = prices.shift(1)
    returns = (prices-shifted_price)/shifted_price

    return returns

project_tests.test_generate_returns(generate_returns)

Tests Passed


C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:154: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only

C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:155: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only



### View Data
Let's generate the closing returns using `generate_returns` and view them using a heatmap.

In [11]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF.  `generate_weighted_returns` creates weighted returns using the returns and weights.

In [12]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    weighted_returns = returns*weights

    return weighted_returns

project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:154: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only

C:\Users\Ayham\Desktop\2021_ND_Programs\2021_AI_ForTrading\Term_1\z_Projects\AI_Finance_Smart-Beta-Portfolio-Optimization\tests.py:155: FutureWarning:

In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only



### View Data
Let's generate the ETF and index returns using `generate_weighted_returns` and view them using a heatmap.

In [13]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

## Cumulative Returns
To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. `calculate_cumulative_returns` calculates the cumulative returns over time given the returns.

In [14]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    # cumulative return = [p(t)/p(0)] -1
    # return was calculated with respected to the previous value

    returns_sum = returns.sum(axis=1) + 1 # add one to multiply the ratios ()
    returns_sum[0] = np.nan # not considered because of the shift in 'returns' 
    cumulative_returns = returns_sum.cumprod(axis= 0,  skipna=True)
    
    return cumulative_returns

project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

Tests Passed


### View Data
Let's generate the ETF and index cumulative returns using `calculate_cumulative_returns` and compare the two.

In [83]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')

## Tracking Error
In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. `tracking_error` returns the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [15]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    delta_return = etf_returns_by_date - benchmark_returns_by_date
    tracking_error = np.sqrt(252)*np.std(delta_return, ddof=1)

    return tracking_error

project_tests.test_tracking_error(tracking_error)

Tests Passed


### View Data
Let's generate the tracking error using `tracking_error`.

In [16]:
smart_beta_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(etf_weighted_returns, 1))
print('Smart Beta Tracking Error: {}'.format(smart_beta_tracking_error))

Smart Beta Tracking Error: 0.10202570304150263


# Part 2: Portfolio Optimization

Create a second portfolio that still reuses the market cap weighted index, but it will be independent of the dividend-weighted portfolio that was created in part 1.

>**Objective**:  We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

**Why?**

One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.


## Covariance
`get_covariance_returns` calculates the covariance of the `returns`. This will be used to calculate the portfolio variance.

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks. [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) is used to get the covariance.  The input is a 2D array in which each row is a stock series, and each column is an observation at the same period of time. `NaN` values are replaced with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [17]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    
    returns_covariance = np.cov(returns.fillna(0).T)

    return returns_covariance

project_tests.test_get_covariance_returns(get_covariance_returns)

Tests Passed


### View Data
Let's look at the covariance generated from `get_covariance_returns`.

In [19]:
covariance_returns = get_covariance_returns(returns)
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

project_helper.plot_covariance_returns_correlation(
    covariance_returns_correlation,
    'Covariance Returns Correlation Matrix')

PlotlyKeyError: Invalid property specified for object of type plotly.graph_objs.Layout: 'xaxis2'

Did you mean "xaxis"?

    Valid properties:
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regardless of this attribute,
            an undefined layout width or height is always
            initialized on the first call to plot.
        a

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

where $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them by calculating the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

### objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio.  If you choose a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.  



### constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [93]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    assert len(covariance_returns.shape) == 2
    assert len(index_weights.shape) == 1
    assert covariance_returns.shape[0] == covariance_returns.shape[1]  == index_weights.shape[0]

    # number of stocks m is number of rows of returns, and also number of index weights
    m = len(index_weights)
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m) 
    
    #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x,covariance_returns)
    
    # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = cvx.norm(x-index_weights)
    
    #objective function
    objective = cvx.Minimize(portfolio_variance + scale * distance_to_index)
    
    #constraints: long and sum to one
    constraints = [x >= 0, sum(x) == 1]

    #use cvxpy to solve the objective
    cvx.Problem(objective, constraints).solve()
    
    #retrieve the weights of the optimized portfolio
    x_values = x.value
    
    return x_values

project_tests.test_get_optimal_weights(get_optimal_weights)

Tests Passed


## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [95]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, index_weights.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

With ETF weights built, we compare it to the index. The next cell calculates the ETF returns and compare it to the index returns.

In [96]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, optim_etf_cumulative_returns, 'Optimized ETF vs Index')

optim_etf_tracking_error = tracking_error(np.sum(index_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

Optimized ETF Tracking Error: 0.05792147628283928


## Rebalance Portfolio Over Time
The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Another strategy is to rebalance the portfolio over the same period instead of using the same weights.  `rebalance_portfolio` rebalances a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, one should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, optoimal weights are computed   using `get_optimal_weights` and `get_covariance_returns`.

In [107]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    for example (from Prateer k: Udacity Forum):
        shift_size = 2 and chunk_size = 4 means that we should rebalance the portfolio every 2 days. 
        While rebalancing, we should look back into the past 4 days of data. 
        So, if we are rebalancing on Oct-17, then we need to look into the data 
        for the days Oct-14 through Oct-17. 
        The next rebalancing should be done on Oct-19 (as the shift_size is 2 days). 
        For the rebalacing on Oct-19, we will be looking into the data from Oct-16 through Oct-19 
        (as the chunk size is 4 days).


    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    all_rebalance_weights = []

    for rebalance_window in range(chunk_size, returns.shape[0], shift_size):

        start_idx = rebalance_window - chunk_size

        local_var_returns = get_covariance_returns(returns.iloc[start_idx:rebalance_window])
        local_optm_weights = get_optimal_weights(local_var_returns, 
                                                index_weights.iloc[rebalance_window-1])
        
        all_rebalance_weights.append(local_optm_weights)

    
    return all_rebalance_weights

project_tests.test_rebalance_portfolio(rebalance_portfolio)

Tests Passed


Run the following cell to rebalance the portfolio using `rebalance_portfolio`.

In [108]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(returns, index_weights, shift_size, chunk_size)

## Portfolio Turnover
With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio.  `get_portfolio_turnover` calculates the annual portfolio turnover. The following formulas will be used:

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [120]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    weights_df = pd.DataFrame(all_rebalance_weights)
    delta_x = (weights_df-weights_df.shift(-1)).fillna(0).values #(xt -xt+1)
    sum_total_turn_over = np.sum(np.abs(delta_x))

    num_rebalance_events = shift_size * rebalance_count
    
    portfolio_turnover = sum_total_turn_over*n_trading_days_in_year/num_rebalance_events
    
    

    return portfolio_turnover

project_tests.test_get_portfolio_turnover(get_portfolio_turnover)

Tests Passed


Run the following cell to get the portfolio turnover from  `get_portfolio turnover`.

In [121]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

16.726832655313835


That's it! You've built a smart beta portfolio in part 1 and did portfolio optimization in part 2. You can now submit your project.